###### Simple script to send a file via the [SWORD](https://en.wikipedia.org/wiki/SWORD_(protocol)) protocol.

In [ ]:
import base64
import hashlib
import logging
import os
import urllib
import xml.etree.ElementTree as ET

In [ ]:
path = ""
authorization = ""
username = ""
password = ""
url = ""

In [ ]:
basic_auth = base64.b64encode(f"{username}:{password}".encode()).decode()

headers = {
    "X-Authorization": authorization,
    "Authorization": f'Basic {basic_auth}',
}

with open(path, "rb") as data:
    md5 = hashlib.md5(data.read()).hexdigest()
    data.seek(0)
    
    file_headers = {
        "Content-MD5": md5,
        "Content-Disposition": f"attachment; filename={os.path.basename(path)}",
        "Content-Type": 'application/zip',
        "Content-Length": os.path.getsize(path)
    }
    
    headers["Content-MD5"] = md5
    request = urllib.request.Request(url=url, data=data, headers={**headers, **file_headers})
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.URLError as e:
        logging.error(f"Request failed: {e.reason}")
        
xml = response.read().decode()
root = ET.fromstring(xml) 
statement_url = root.find(".//*[@rel='http://purl.org/net/sword/terms/statement']").attrib["href"]
print(f"Statement URL is: {statement_url}")

request = urllib.request.Request(url=statement_url, headers=headers)
try:
    response = urllib.request.urlopen(request)
except urllib.error.URLError as e:
    logging.error(f"Request failed: {e.reason}")

print(response.read().decode())